In [1]:
try:
    dash_thread.stop()
    print("Stopped old Dash server.")
except NameError:
    pass

In [1]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path.cwd() / "data captures"
files = sorted(DATA_DIR.glob("comfort_log_*.csv"))
print("Found", len(files), "files in", DATA_DIR)

if files:
    latest = files[-1]
    print("Latest:", latest.name)
    df = pd.read_csv(latest, nrows=8)
    display(df.head())
else:
    print("No files yet – keep your logger running.")


Found 4 files in C:\Users\DELL\data captures
Latest: comfort_log_20251003_0158.csv


,ms_since_start,tempC,humidity,accel_rms,iso_time
0,2007,15.3,72.2,0.0284,2025-10-03T01:58:42
1,2507,15.3,72.2,0.0000,2025-10-03T01:58:42
2,3007,15.3,72.2,0.0000,2025-10-03T01:58:43
3,3507,15.3,72.2,0.0000,2025-10-03T01:58:43
4,4007,15.4,72.2,0.0000,2025-10-03T01:58:44


In [3]:
from dash import Dash, dcc, html, Output, Input
import plotly.graph_objects as go
import pandas as pd
from pathlib import Path
import os, glob, socket, threading
from werkzeug.serving import make_server
from IPython.display import HTML, display

DATA_DIR = Path.cwd() / "data captures"
DATA_DIR.mkdir(exist_ok=True)

T_MIN, T_MAX = 18.0, 25.0
H_MIN, H_MAX = 40.0, 70.0
VIBE_LIGHT = 0.08  # g

def latest_csv():
    files = sorted(glob.glob(str(DATA_DIR / "comfort_log_*.csv")))
    return files[-1] if files else None

app = Dash(__name__)
app.title = "Comfort Monitor"
app.layout = html.Div([
    html.H2("Comfort-while-studying Monitor"),
    html.Div(id="file-indicator", style={"margin":"6px 0"}),
    html.Div(id="status-chip", style={"fontWeight":"bold","fontSize":"20px","marginBottom":"12px"}),
    dcc.Graph(id="temp-graph"),
    dcc.Graph(id="hum-graph"),
    dcc.Graph(id="vibe-graph"),
    dcc.Interval(id="tick", interval=2000, n_intervals=0),
])

@app.callback(
    Output("file-indicator", "children"),
    Output("status-chip", "children"),
    Output("temp-graph", "figure"),
    Output("hum-graph", "figure"),
    Output("vibe-graph", "figure"),
    Input("tick", "n_intervals"),
)
def refresh(_):
    empty = go.Figure()
    path = latest_csv()
    if not path or not os.path.exists(path):
        return ("No CSV found in 'data captures' yet. Keep the logger running.",
                "Waiting for data…", empty, empty, empty)

    df = pd.read_csv(path)

    required = {"ms_since_start","tempC","humidity","accel_rms"}
    if not required.issubset(df.columns):
        return (f"Bad CSV columns: {os.path.basename(path)}",
                f"Expect {sorted(required)}",
                empty, empty, empty)

    df = df.rename(columns={"ms_since_start":"ms"})
    df["sec"] = pd.to_numeric(df["ms"], errors="coerce")/1000.0
    for c in ["tempC","humidity","accel_rms"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    df["tempC_s"]     = df["tempC"].rolling(5, min_periods=1).mean()
    df["humidity_s"]  = df["humidity"].rolling(5, min_periods=1).mean()
    df["accel_rms_s"] = df["accel_rms"].rolling(5, min_periods=1).mean()

    latest = df.iloc[-1]
    comfort_ok = (T_MIN <= latest["tempC_s"] <= T_MAX) and \
                 (H_MIN <= latest["humidity_s"] <= H_MAX) and \
                 (latest["accel_rms_s"] <= VIBE_LIGHT)
    status = "✅ Comfort OK" if comfort_ok else "⚠️ Action needed (adjust temp/humidity or take a short break)"

    f1 = go.Figure()
    f1.add_trace(go.Scatter(x=df["sec"], y=df["tempC_s"], mode="lines", name="Temp °C"))
    f1.add_hline(y=T_MIN); f1.add_hline(y=T_MAX)
    f1.update_layout(title="Temperature (°C)", xaxis_title="Seconds", yaxis_title="°C")

    f2 = go.Figure()
    f2.add_trace(go.Scatter(x=df["sec"], y=df["humidity_s"], mode="lines", name="RH %"))
    f2.add_hline(y=H_MIN); f2.add_hline(y=H_MAX)
    f2.update_layout(title="Humidity (%)", xaxis_title="Seconds", yaxis_title="%RH")

    f3 = go.Figure()
    f3.add_trace(go.Scatter(x=df["sec"], y=df["accel_rms_s"], mode="lines", name="Vibration RMS (g)"))
    f3.add_hline(y=VIBE_LIGHT)
    f3.update_layout(title="Desk Vibration RMS (g)", xaxis_title="Seconds", yaxis_title="g (RMS)")

    return (f"Reading: {os.path.basename(path)}", status, f1, f2, f3)

def free_port(start=8050, tries=20):
    import socket
    for p in range(start, start+tries):
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            if s.connect_ex(("127.0.0.1", p)) != 0:
                return p
    raise RuntimeError("No free port")

PORT = free_port(8050)
HOST = "127.0.0.1"

class DashServerThread(threading.Thread):
    def __init__(self, app, host, port):
        super().__init__(daemon=True)
        self.srv = make_server(host, port, app.server)
    def run(self):
        self.srv.serve_forever()
    def stop(self):
        self.srv.shutdown()

dash_thread = DashServerThread(app, HOST, PORT)
dash_thread.start()

display(HTML(f"""
<div style="padding:10px;border:1px solid #ccc;border-radius:8px;">
  Dash server running at
  <a href="http://{HOST}:{PORT}" target="_blank">http://{HOST}:{PORT}</a>
</div>
"""))
